In [1]:
import json
import os
import PIL.Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms

from torchvision.datasets import CocoDetection

from architectures.basic import BoundingBoxModel as BasicBoundingBoxModel
from architectures.bigger_basic import BoundingBoxModel as BiggerBasicBoundingBoxModel
from functions.drawing import draw_rectangle
from functions.losses import CIoULoss
from functions.mapping import extract_bboxes

print(torch.__version__)
print(torch.cuda.is_available())

2.3.1
True


In [2]:
root = os.path.join("data", "test_images")
ann_file = os.path.join("data", "test_coco_annotations.json")
transform = transforms.Compose([
    transforms.ToTensor()
])
dataset = CocoDetection(
    root=root,
    annFile=ann_file,
    transform=transform
)
test_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    prefetch_factor=16
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
cuda


In [3]:
variations = {
    "1": BasicBoundingBoxModel,
    "2": BiggerBasicBoundingBoxModel,
    "3": BiggerBasicBoundingBoxModel
}
model_metrics = {
    "1": [],
    "2": [],
    "3": []
}

In [4]:
if not os.path.exists(os.path.join("testing_steps", "best_variations.json")):
    with torch.no_grad():
        for filename in os.listdir("checkpoints"):
            if filename.endswith(".pt"):
                parts = filename.split("_")
                model_variation = parts[1]
                run_index = parts[5].split(".")[0]
                model_path = os.path.join("checkpoints", filename)
                model = variations[model_variation]().to(device)
                smooth_l1_loss_f = nn.SmoothL1Loss()
                ciou_loss_f = CIoULoss()
                
                model.load_state_dict(torch.load(model_path, map_location=device))
            
                metrics = []
                
                for data in test_loader:
                    inputs, true_boxes = data
                    true_boxes = torch.stack(extract_bboxes(true_boxes)).to(device)
                    inputs = inputs.to(device)
                    pred_boxes = model(inputs)
                    l1_metric = smooth_l1_loss_f(pred_boxes, true_boxes)
                    ciou_loss = ciou_loss_f(pred_boxes, true_boxes)
            
                    metrics.append((l1_metric, ciou_loss))
    
                mean_l1_metric = sum(map(lambda x: x[0], metrics)) / len(metrics)
                mean_ciou_metric = sum(map(lambda x: x[1], metrics)) / len(metrics)
    
                model_metrics[model_variation].append((mean_l1_metric, mean_ciou_metric))

In [5]:
if not os.path.exists(os.path.join("testing_steps", "best_variations.json")):
    best_variations = {
        "1": [],
        "2": [],
        "3": []
    }
    
    for namespaced_key, tensors in model_metrics.items():
        l1_items = list(map(lambda x: x[0].item(), tensors))
        ciou_items = list(map(lambda x: x[1].item(), tensors))
        min_l1 = min(l1_items)
        min_ciou = min(ciou_items)
        min_l1_idx = l1_items.index(min_l1)
        min_ciou_idx = ciou_items.index(min_ciou)
        variation_number_l1 = min_l1_idx + 1
        variation_number_ciou = min_ciou_idx + 1
        
        best_variations[namespaced_key].append({
            f"{str(variation_number_l1)}_l1": min_l1, 
            f"{str(variation_number_ciou)}_ciou": min_ciou
        })
    
    with open(os.path.join("testing_steps", "best_variations.json"), "w") as json_file:
        json.dump(best_variations, json_file)

In [6]:
if os.path.exists(os.path.join("testing_steps", "best_variations.json")):
    with open(os.path.join("testing_steps", "best_variations.json"), "r") as json_file:
        best_variations = json.load(json_file) 

In [11]:
materialized_variations = []

for namespaced_key, variation_numbers in best_variations.items():
    key = namespaced_key.split("_")[0]
    model_cls = variations[key]

    for filename in os.listdir("checkpoints"):
        for namespaced_variation_number in variation_numbers[0]:
            variation_number = namespaced_variation_number.split("_")[0]
            
            if filename.startswith(f"checkpoint_{key}") and filename.endswith(f"_run_{variation_number}.pt"):
                model = model_cls().to(device)
                model_path = os.path.join("checkpoints", filename)
                
                model.load_state_dict(torch.load(model_path, map_location=device))
                materialized_variations.append((
                    # model, 
                    key, 
                    variation_number, 
                    variation_numbers[0][namespaced_variation_number]
                ))  

The best model turns out to be the bigger one. It has the lowest SmoothL1Loss function result, as well as lowest CIoU function result.

In [8]:
print(materialized_variations[2][3])
print(materialized_variations[3][3])

5.780552387237549
0.22702017426490784


In [9]:
best_model = materialized_variations[2][0]
out_dir = os.path.join("data", "output_phase1")

best_model.eval()
os.makedirs(out_dir, exist_ok=True)

In [10]:
test_loader_small_batch = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    prefetch_factor=16
)

with torch.no_grad():
    for idx, (images, targets) in enumerate(test_loader_small_batch):
        images = images.to(device)
        outputs = model(images)
        img_id = dataset.ids[idx]
        img_info = dataset.coco.loadImgs(img_id)[0]
        img_path = img_info["file_name"]
        orig_img = PIL.Image.open(os.path.join(root, img_path))

        for output in outputs:
            coords = tuple(map(int, output[:4]))
            output_path = os.path.join(out_dir, os.path.basename(img_path))
            draw_rectangle(orig_img, coords, output_path)